# Model Building

In this notebook, we will be building ML models to predict whether a person is earning more or less than $50K.

The data used is the open-source [Census Dataset](https://archive.ics.uci.edu/ml/datasets/census+income), obtained from UCL ML Repository.

## Setup

First, we import required libraries and read the data.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from xgboost import XGBClassifier
from pickle import dump
from pickle import load

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Read in data
df = pd.read_csv('../data/census_intermediate.csv')

In [3]:
# Display snippet of data
df.head(5)

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Data Preprocessing

Next, we perform some data preprocessing, to get the data ready for ML Modelling.

In [4]:
# Encode categorical features and save encoder
LE_Features = ['workclass', 'marital-status', 'occupation', 
               'relationship', 'race','sex', 'native-country']

for feature in LE_Features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    
    mapping = dict(zip(le.classes_, range(len(le.classes_))))
    print(feature)
    print(mapping)
    filepath = '../model/{}_encoder.pkl'.format(feature)
    #dump(le, open(filepath, 'wb'))
    print('Saved encoder to model/', '\n')

workclass
{'?': 0, 'Federal-gov': 1, 'Local-gov': 2, 'Never-worked': 3, 'Private': 4, 'Self-emp-inc': 5, 'Self-emp-not-inc': 6, 'State-gov': 7, 'Without-pay': 8}
Saved encoder to model/ 

marital-status
{'Divorced': 0, 'Married-AF-spouse': 1, 'Married-civ-spouse': 2, 'Married-spouse-absent': 3, 'Never-married': 4, 'Separated': 5, 'Widowed': 6}
Saved encoder to model/ 

occupation
{'?': 0, 'Adm-clerical': 1, 'Armed-Forces': 2, 'Craft-repair': 3, 'Exec-managerial': 4, 'Farming-fishing': 5, 'Handlers-cleaners': 6, 'Machine-op-inspct': 7, 'Other-service': 8, 'Priv-house-serv': 9, 'Prof-specialty': 10, 'Protective-serv': 11, 'Sales': 12, 'Tech-support': 13, 'Transport-moving': 14}
Saved encoder to model/ 

relationship
{'Husband': 0, 'Not-in-family': 1, 'Other-relative': 2, 'Own-child': 3, 'Unmarried': 4, 'Wife': 5}
Saved encoder to model/ 

race
{'Amer-Indian-Eskimo': 0, 'Asian-Pac-Islander': 1, 'Black': 2, 'Other': 3, 'White': 4}
Saved encoder to model/ 

sex
{' Female': 0, ' Male': 1}
Sa

In [5]:
# Display snippet of data
df.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,7,77516,Bachelors,13,4,1,1,4,1,2174,0,40,39,<=50K
1,50,6,83311,Bachelors,13,2,4,0,4,1,0,0,13,39,<=50K
2,38,4,215646,HS-grad,9,0,6,1,4,1,0,0,40,39,<=50K
3,53,4,234721,11th,7,2,6,0,2,1,0,0,40,39,<=50K
4,28,4,338409,Bachelors,13,2,10,5,2,0,0,0,40,5,<=50K


In [6]:
# Drop unnecessary columns
df.drop('education', axis = 1, inplace = True)

In [7]:
# Encode target feature
df['salary'] = df['salary'].replace({'<=50K': 0, '>50K':1})

## ML Modelling

After preprocessing, the data will be ready for ML development.

In [8]:
# Get X and y features
y = df['salary']
X = df.drop('salary', axis = 1)

In [9]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [10]:
# Scale X features
scaler = RobustScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Save scaler
filepath = '../model/robust_scaler.pkl'
#dump(scaler, open(filepath, 'wb'))

In [12]:
# Initial model - Logistic regression
Log_model = LogisticRegression(max_iter = 1000).fit(X_train, y_train)
y_pred_lr = Log_model.predict(X_test)
accuracy_score(y_test, y_pred_lr)

0.8284968524489482

In [13]:
# Get classification report
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      5026
           1       0.69      0.45      0.54      1487

    accuracy                           0.83      6513
   macro avg       0.77      0.69      0.72      6513
weighted avg       0.82      0.83      0.81      6513



In [14]:
# Hyperparameter tuning for logistic regression model (with L2 regularization)
Log_model = LogisticRegression(max_iter = 1000)

grid_values = {'C':[0.001,0.01, 0.1, 1, 10, 100, 1000, 10000], 'penalty': ['l2']}

cross_validation = StratifiedKFold(n_splits = 5)

grid_log_model = GridSearchCV(Log_model, param_grid = grid_values, scoring = 'accuracy', cv = cross_validation)
grid_log_model.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
                         'penalty': ['l2']},
             scoring='accuracy')

In [15]:
# Get best params for ridge regression model
grid_log_model.best_params_

{'C': 10000, 'penalty': 'l2'}

In [16]:
# Get classification model for best ridge regression model
y_pred_lr = grid_log_model.predict(X_test)
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      5026
           1       0.70      0.45      0.55      1487

    accuracy                           0.83      6513
   macro avg       0.77      0.70      0.72      6513
weighted avg       0.82      0.83      0.82      6513



In [17]:
# Hyperparameter tuning for XGBoost model
grid_values = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                'n_estimators' : [2, 3, 5, 10, 15, 20, 30, 50]}

xgb = XGBClassifier()

cross_validation = StratifiedKFold(n_splits = 5)

grid_xgb_model = GridSearchCV(xgb, param_grid = grid_values,scoring = "accuracy", cv = cross_validation)
grid_xgb_model.fit(X_train, y_train)

print("Best parameters found: ", grid_xgb_model.best_params_)

Best parameters found:  {'max_depth': 6, 'n_estimators': 30}


In [18]:
# Get classification report for best XGBoost model
y_pred_xgb = grid_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      5026
           1       0.75      0.66      0.70      1487

    accuracy                           0.87      6513
   macro avg       0.83      0.80      0.81      6513
weighted avg       0.87      0.87      0.87      6513



## Final Training

We train the chosen model on the entire dataset, and then save it for future use.

In [19]:
# Chosen model
xgb_final = XGBClassifier(max_depth = 6, n_estimators = 30)
xgb_final.fit(X_train, y_train)

y_preds = xgb_final.predict(X_test)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      5026
           1       0.75      0.66      0.70      1487

    accuracy                           0.87      6513
   macro avg       0.83      0.80      0.81      6513
weighted avg       0.87      0.87      0.87      6513



In [20]:
# Save model
filepath = '../model/xgb_model.pkl'
#dump(xgb_final, open(filepath, 'wb'))

## Model Inference

Below, we show how to use the trained model for prediction.

In [21]:
# Get data for prediction
df = pd.read_csv('../data/census_intermediate.csv')

In [22]:
# Preprocessing target feature
df['salary'] = df['salary'].replace({'<=50K': 0, '>50K':1})

In [23]:
# Get encoder for categorical features' preprocessing
LE_Features = ['workclass', 'marital-status', 'occupation', 
               'relationship', 'race','sex', 'native-country']

for feature in LE_Features:
    filepath = '../model/{}_encoder.pkl'.format(feature)
    le = load(open(filepath, 'rb'))
    df[feature] = le.transform(df[feature])
    print('Encoding for {}: ... done'.format(feature))

Encoding for workclass: ... done
Encoding for marital-status: ... done
Encoding for occupation: ... done
Encoding for relationship: ... done
Encoding for race: ... done
Encoding for sex: ... done
Encoding for native-country: ... done


In [24]:
# Split data
y = df['salary']
X = df.drop(['salary', 'education'], axis = 1)

In [25]:
# Get scaler and preprocess numerical features
filepath = '../model/robust_scaler.pkl'
scaler = load(open(filepath, 'rb'))

X = scaler.transform(X)

In [26]:
# Get model for inference
filepath = '../model/xgb_model.pkl'
model = load(open(filepath, 'rb'))

y_preds = model.predict(X)
print(classification_report(y_preds, y))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92     25886
           1       0.68      0.80      0.74      6675

    accuracy                           0.88     32561
   macro avg       0.81      0.85      0.83     32561
weighted avg       0.89      0.88      0.89     32561



In [27]:
# Function to combine all preprocessing activities
def process_data(data_df):
    data_df['salary'] = data_df['salary'].replace({'<=50K': 0, '>50K':1})
    LE_Features = ['workclass', 'marital-status', 'occupation', 
                   'relationship', 'race','sex', 'native-country']

    for feature in LE_Features:
        filepath = '../model/{}_encoder.pkl'.format(feature)
        le = load(open(filepath, 'rb'))
        data_df[feature] = le.transform(data_df[feature])
        #print('Encoding for {}: ... done'.format(feature))
        
    y = data_df['salary']
    X = data_df.drop(['salary', 'education'], axis = 1)
    
    filepath = '../model/robust_scaler.pkl'
    scaler = load(open(filepath, 'rb'))

    X = scaler.transform(X)
    
    return X,y

In [28]:
# Store predicted values
df['Predicted'] = y_preds

In [29]:
# Save data & prediction results
#df.to_csv('../data/Predictions.csv', index = False)

## Assessing Model Performance on Slices

Below, we analyse our XGBoost model performance on data slices based on categorical features.

In [46]:
# Get data
df = pd.read_csv('../data/census_intermediate.csv')

In [31]:
# Split X & y
y = df['salary']
X = df.drop('salary', axis = 1)

In [32]:
# Split training & testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [33]:
# Ensure test subset has all categories
LE_Features = ['workclass', 'education', 'marital-status', 'occupation', 
               'relationship', 'race','sex', 'native-country']

for feature in LE_Features:
    print(set(df[feature]))
    print(len(set(df[feature])))
    print(set(X_test[feature]))
    print(len(set(X_test[feature])))
    print('\n')

{'Self-emp-inc', '?', 'Without-pay', 'Self-emp-not-inc', 'Private', 'Never-worked', 'State-gov', 'Local-gov', 'Federal-gov'}
9
{'Self-emp-inc', '?', 'Without-pay', 'Self-emp-not-inc', 'Private', 'Never-worked', 'State-gov', 'Local-gov', 'Federal-gov'}
9


{'Assoc-acdm', 'Prof-school', '11th', '9th', '10th', 'Masters', 'Some-college', '7th-8th', 'Assoc-voc', 'Preschool', 'HS-grad', 'Doctorate', 'Bachelors', '12th', '1st-4th', '5th-6th'}
16
{'Assoc-acdm', 'Prof-school', '11th', '10th', '9th', '7th-8th', 'Some-college', 'Masters', 'Assoc-voc', 'Preschool', 'HS-grad', 'Doctorate', 'Bachelors', '12th', '1st-4th', '5th-6th'}
16


{'Married-spouse-absent', 'Never-married', 'Divorced', 'Married-civ-spouse', 'Widowed', 'Married-AF-spouse', 'Separated'}
7
{'Married-spouse-absent', 'Never-married', 'Divorced', 'Married-civ-spouse', 'Widowed', 'Married-AF-spouse', 'Separated'}
7


{'Craft-repair', 'Tech-support', '?', 'Sales', 'Machine-op-inspct', 'Protective-serv', 'Transport-moving', 'Handlers-c

In [34]:
# Data subset to be used for assessing model performance
data = X_test.join(y_test)

In [35]:
# Snippet of data
data

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
9646,62,Self-emp-not-inc,26911,7th-8th,4,Widowed,Other-service,Not-in-family,White,Female,0,0,66,United-States,<=50K
709,18,Private,208103,11th,7,Never-married,Other-service,Other-relative,White,Male,0,0,25,United-States,<=50K
7385,25,Private,102476,Bachelors,13,Never-married,Farming-fishing,Own-child,White,Male,27828,0,50,United-States,>50K
16671,33,Private,511517,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<=50K
21932,36,Private,292570,11th,7,Never-married,Machine-op-inspct,Unmarried,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5889,39,Private,146091,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,20,United-States,>50K
25723,17,Private,347322,10th,6,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K
29514,35,Private,290226,HS-grad,9,Never-married,Transport-moving,Own-child,White,Male,0,0,40,United-States,<=50K
1600,30,Private,27207,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,45,United-States,<=50K


In [36]:
# Prepare df to store model performance metrics on data slices
cols = ['Feature', 'Slice', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
slice_performance_df = pd.DataFrame(columns = cols)

In [37]:
# Get performance on each data slice
filepath = '../model/xgb_model.pkl'
model = load(open(filepath, 'rb'))

Cat_Features = ['workclass', 'education', 'marital-status', 'occupation', 
               'relationship', 'race','sex', 'native-country']

for feature in Cat_Features:
    for slc in data[feature].unique():
        df_slice = data[data[feature] == slc]
        X_slice, y_slice = process_data(df_slice)
        y_pred_slice = model.predict(X_slice)
        acc = accuracy_score(y_pred_slice, y_slice)
        prec = precision_score(y_pred_slice, y_slice, zero_division = 0)
        rec = recall_score(y_pred_slice, y_slice, zero_division = 0)
        f1 = f1_score(y_pred_slice, y_slice, zero_division = 0)
        
        row_data = {'Feature':feature, 'Slice':slc, 'Accuracy':acc,
                    'Precision':prec, 'Recall':rec, 'F1 Score':f1}
        slice_performance_df = slice_performance_df.append(row_data,
                                                           ignore_index = True)


In [38]:
# Store results
path = '../slice_output.txt'

#export DataFrame to text file
#with open(path, 'a') as f:
#    df_string = slice_performance_df.to_string(header=True, index=False)
#    f.write(df_string)